In [39]:
import time
import random
import pandas as pd
import os

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# ==========================
# CONFIG
# ==========================

BASE_URLS = {
    "Season_7": "https://results.hyrox.com/season-7/?pid=start",
    "Season_8": "https://results.hyrox.com/season-8/"
}

SAVE_ROOT = r"Datasets\Hyrox"

DIVISION_OPTIONS = [
    "HYROX PRO",
    "HYROX PRO - Overall",
    "HYROX PRO DOUBLES",
    "HYROX PRO DOUBLES - Overall"
]

def human_pause(a=3, b=7):
    time.sleep(random.uniform(a, b))

# ==========================
# DRIVER
# ==========================

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/120.0.0.0 Safari/537.36"
)

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 25)

# ==========================
# SCRAPE PAGES FUNCTION
# ==========================

def scrape_pages(race_name, division, gender, race_results):
    page_number = 1
    is_doubles = "DOUBLES" in division.upper()

    while True:
        try:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f"div[data-sex='{gender[0]}']")))
        except TimeoutException:
            print("      No results container found")
            return

        human_pause(2, 5)

        try:
            container = driver.find_element(By.CSS_SELECTOR, f"div[data-sex='{gender[0]}']")
        except NoSuchElementException:
            print("      No container for this gender")
            return

        rows = container.find_elements(By.CSS_SELECTOR, "li.list-group-item.row")
        if not rows:
            print("      No rows found in container")
            return

        for row in rows:
            if "list-group-header" in row.get_attribute("class"):
                continue
            try:
                rank = row.find_element(By.CSS_SELECTOR, ".place-primary").text
                age_rank = row.find_element(By.CSS_SELECTOR, ".place-secondary").text
                total_time = row.find_element(By.CSS_SELECTOR, ".type-time").text.replace("Total", "").strip()
                age_group = row.find_element(By.CSS_SELECTOR, ".type-age_class").text.replace("Age Group", "").strip()

                if is_doubles:
                    # For DOUBLES, get member names
                    members = row.find_elements(By.CSS_SELECTOR, ".type-relay_member a")
                    member_names = " & ".join([m.text for m in members])
                    race_results.append([race_name, division, gender, rank, age_rank, member_names, "", age_group, total_time])
                else:
                    # Singles
                    name = row.find_element(By.CSS_SELECTOR, "h4.type-fullname").text
                    nation = row.find_element(By.CSS_SELECTOR, ".nation__abbr").text
                    race_results.append([race_name, division, gender, rank, age_rank, name, nation, age_group, total_time])

            except:
                continue

        print(f"      Page {page_number} scraped")
        human_pause(3, 6)

        try:
            next_button = driver.find_element(By.XPATH, "//a[text()='>']")
            driver.execute_script("arguments[0].click();", next_button)
            page_number += 1
            human_pause(5, 10)
        except NoSuchElementException:
            print("      Last page reached")
            break

# ==========================
# MAIN LOOP
# ==========================

for season, base_url in BASE_URLS.items():
    print(f"\n==============================\nSTARTING {season}\n==============================\n")
    season_folder = os.path.join(SAVE_ROOT, season)
    os.makedirs(season_folder, exist_ok=True)

    driver.get(base_url)
    human_pause(5, 10)

    try:
        wait.until(EC.presence_of_element_located((By.ID, "default-lists-event_main_group")))
    except TimeoutException:
        print(f"No races found for {season}")
        continue

    race_dropdown = Select(driver.find_element(By.ID, "default-lists-event_main_group"))
    races = [(opt.text, opt.get_attribute("value")) for opt in race_dropdown.options]
    print(f"Found {len(races)} races in {season}")

    for race_name, race_value in races:
        safe_name = race_name.replace(" ", "_").replace("/", "-")
        print(f"\nProcessing race: {race_name}")

        # ==========================
        # Check available divisions
        # ==========================
        driver.get(base_url)
        human_pause(5, 9)
        Select(driver.find_element(By.ID, "default-lists-event_main_group")).select_by_value(race_value)
        human_pause(3, 6)
        select_division = Select(driver.find_element(By.ID, "default-lists-event"))
        available_divisions = [o.text for o in select_division.options]
        valid_divisions = [d for d in DIVISION_OPTIONS if d in available_divisions]

        if not valid_divisions:
            print("   No valid HYROX PRO divisions found for this race")
            continue

        for div in valid_divisions:
            div_safe_name = div.replace(" ", "_").replace("/", "-")
            file_path_div = os.path.join(season_folder, f"{safe_name}_{div_safe_name}.csv")

            if os.path.exists(file_path_div):
                print(f"   Division CSV already exists: {file_path_div}, skipping division")
                continue  # skip this division entirely

            print(f"   Division: {div}")
            division_results = []

            for gender_code, gender_name in [("M", "Men"), ("W", "Women")]:
                print(f"      Gender: {gender_name}")

                # Go back to base page and re-select race + division
                driver.get(base_url)
                human_pause(4, 8)
                Select(driver.find_element(By.ID, "default-lists-event_main_group")).select_by_value(race_value)
                human_pause(2, 5)
                Select(driver.find_element(By.ID, "default-lists-event")).select_by_visible_text(div)
                human_pause(2, 5)

                # Select gender and number of results
                Select(driver.find_element(By.ID, "default-lists-sex")).select_by_value(gender_code)
                Select(driver.find_element(By.ID, "default-num_results")).select_by_value("100")
                human_pause(2, 5)

                # Submit form
                driver.find_element(By.ID, "default-submit").click()
                human_pause(5, 10)

                # Scrape results
                scrape_pages(race_name, div, gender_name, division_results)
                human_pause(2, 5)

            # Save CSV after both genders are scraped
            columns = ["Race", "Division", "Gender", "Rank Overall", "Rank Age Group",
                       "Name", "Nation", "Age Group", "Total Time"]
            df_div = pd.DataFrame(division_results, columns=columns)
            df_div.to_csv(file_path_div, index=False)
            print(f"   Saved {len(df_div)} rows to {file_path_div}")
            human_pause(3, 6)

print("\nALL DONE.")
driver.quit()


STARTING Season_7

Found 73 races in Season_7

Processing race: 2025 Shanghai
   Division CSV already exists: Datasets\Hyrox\Season_7\2025_Shanghai_HYROX_PRO.csv, skipping division
   Division CSV already exists: Datasets\Hyrox\Season_7\2025_Shanghai_HYROX_PRO_DOUBLES.csv, skipping division

Processing race: 2025 Atlanta
   Division CSV already exists: Datasets\Hyrox\Season_7\2025_Atlanta_HYROX_PRO_-_Overall.csv, skipping division
   Division CSV already exists: Datasets\Hyrox\Season_7\2025_Atlanta_HYROX_PRO_DOUBLES_-_Overall.csv, skipping division

Processing race: 2025 Valencia
   Division CSV already exists: Datasets\Hyrox\Season_7\2025_Valencia_HYROX_PRO_-_Overall.csv, skipping division
   Division CSV already exists: Datasets\Hyrox\Season_7\2025_Valencia_HYROX_PRO_DOUBLES_-_Overall.csv, skipping division

Processing race: 2025 Maastricht
   Division CSV already exists: Datasets\Hyrox\Season_7\2025_Maastricht_HYROX_PRO.csv, skipping division
   Division CSV already exists: Dataset

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=145.0.7632.76); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff66ae0aa55
	0x7ff66ab68630
	0x7ff66a8fd75d
	0x7ff66a8e8c72
	0x7ff66a90f015
	0x7ff66a9890e0
	0x7ff66a9a5282
	0x7ff66a949098
	0x7ff66a949f83
	0x7ff66ae37810
	0x7ff66ae31afd
	0x7ff66ae52c1a
	0x7ff66ab83345
	0x7ff66ab8b81c
	0x7ff66ab71924
	0x7ff66ab71ad6
	0x7ff66ab57e47
	0x7ff8b1bae8d7
	0x7ff8b33cc40c


In [41]:
import time
import random
import pandas as pd
import os

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# ==========================
# CONFIG
# ==========================

BASE_URLS = {
    "Season_7": "https://results.hyrox.com/season-7/?pid=start",
    "Season_8": "https://results.hyrox.com/season-8/"
}

SAVE_ROOT = r"Datasets\Hyrox"

DIVISION_OPTIONS = [
    "HYROX PRO",
    "HYROX PRO - Overall",
    "HYROX PRO DOUBLES",
    "HYROX PRO DOUBLES - Overall"
]

def human_pause(a=1, b=3):
    time.sleep(random.uniform(a, b))

# ==========================
# DRIVER
# ==========================

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/120.0.0.0 Safari/537.36"
)

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 25)

# ==========================
# SCRAPE PAGES FUNCTION
# ==========================

def scrape_pages(race_name, division, gender, race_results):
    page_number = 1
    is_doubles = "DOUBLES" in division.upper()

    while True:
        try:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f"div[data-sex='{gender[0]}']")))
        except TimeoutException:
            print("      No results container found")
            return

        human_pause(1, 2)

        try:
            container = driver.find_element(By.CSS_SELECTOR, f"div[data-sex='{gender[0]}']")
        except NoSuchElementException:
            print("      No container for this gender")
            return

        rows = container.find_elements(By.CSS_SELECTOR, "li.list-group-item.row")
        if not rows:
            print("      No rows found in container")
            return

        for row in rows:
            if "list-group-header" in row.get_attribute("class"):
                continue
            try:
                rank = row.find_element(By.CSS_SELECTOR, ".place-primary").text
                age_rank = row.find_element(By.CSS_SELECTOR, ".place-secondary").text
                total_time = row.find_element(By.CSS_SELECTOR, ".type-time").text.replace("Total", "").strip()
                age_group = row.find_element(By.CSS_SELECTOR, ".type-age_class").text.replace("Age Group", "").strip()

                if is_doubles:
                    members = row.find_elements(By.CSS_SELECTOR, ".type-relay_member a")
                    member_names = " & ".join([m.text for m in members])
                    race_results.append([race_name, division, gender, rank, age_rank, member_names, "", age_group, total_time])
                else:
                    name = row.find_element(By.CSS_SELECTOR, "h4.type-fullname").text
                    nation = row.find_element(By.CSS_SELECTOR, ".nation__abbr").text
                    race_results.append([race_name, division, gender, rank, age_rank, name, nation, age_group, total_time])

            except:
                continue

        print(f"      Page {page_number} scraped")
        human_pause(1, 2)

        try:
            next_button = driver.find_element(By.XPATH, "//a[text()='>']")
            driver.execute_script("arguments[0].click();", next_button)
            page_number += 1
            human_pause(2, 4)
        except NoSuchElementException:
            print("      Last page reached")
            break

# ==========================
# MAIN LOOP
# ==========================

for season, base_url in BASE_URLS.items():
    print(f"\n==============================\nSTARTING {season}\n==============================\n")
    season_folder = os.path.join(SAVE_ROOT, season)
    os.makedirs(season_folder, exist_ok=True)

    # ===== Build set of already downloaded CSVs =====
    existing_csvs = set(os.listdir(season_folder))

    driver.get(base_url)
    human_pause(2, 4)

    try:
        wait.until(EC.presence_of_element_located((By.ID, "default-lists-event_main_group")))
    except TimeoutException:
        print(f"No races found for {season}")
        continue

    race_dropdown = Select(driver.find_element(By.ID, "default-lists-event_main_group"))
    races = [(opt.text, opt.get_attribute("value")) for opt in race_dropdown.options]
    print(f"Found {len(races)} races in {season}")

    # ===== Build list of race/division combos that still need scraping =====
    races_to_scrape = []
    for race_name, race_value in races:
        safe_name = race_name.replace(" ", "_").replace("/", "-")
        driver.get(base_url)
        human_pause(1, 2)
        Select(driver.find_element(By.ID, "default-lists-event_main_group")).select_by_value(race_value)
        human_pause(1, 2)
        select_division = Select(driver.find_element(By.ID, "default-lists-event"))
        available_divisions = [o.text for o in select_division.options]
        valid_divisions = [d for d in DIVISION_OPTIONS if d in available_divisions]

        # Check which divisions still need scraping
        divisions_needed = [d for d in valid_divisions
                            if f"{safe_name}_{d.replace(' ', '_').replace('/', '-')}.csv" not in existing_csvs]

        if divisions_needed:
            races_to_scrape.append((race_name, race_value, divisions_needed))
        else:
            print(f"   Skipping {race_name}, all divisions already downloaded")

    print(f"\n{len(races_to_scrape)} races need scraping in {season}")

    # ===== Scrape only the races that need it =====
    for race_name, race_value, divisions_needed in races_to_scrape:
        safe_name = race_name.replace(" ", "_").replace("/", "-")
        print(f"\nProcessing race: {race_name}")

        for div in divisions_needed:
            div_safe_name = div.replace(" ", "_").replace("/", "-")
            file_path_div = os.path.join(season_folder, f"{safe_name}_{div_safe_name}.csv")
            print(f"   Division: {div}")
            division_results = []

            for gender_code, gender_name in [("M", "Men"), ("W", "Women")]:
                print(f"      Gender: {gender_name}")

                # Go back to base page and re-select race + division
                driver.get(base_url)
                human_pause(1, 2)
                Select(driver.find_element(By.ID, "default-lists-event_main_group")).select_by_value(race_value)
                human_pause(1, 2)
                Select(driver.find_element(By.ID, "default-lists-event")).select_by_visible_text(div)
                human_pause(1, 2)

                # Select gender and number of results
                Select(driver.find_element(By.ID, "default-lists-sex")).select_by_value(gender_code)
                Select(driver.find_element(By.ID, "default-num_results")).select_by_value("100")
                human_pause(1, 2)

                # Submit form
                driver.find_element(By.ID, "default-submit").click()
                human_pause(2, 4)

                # Scrape results
                scrape_pages(race_name, div, gender_name, division_results)
                human_pause(1, 2)

            # Save CSV after both genders are scraped
            columns = ["Race", "Division", "Gender", "Rank Overall", "Rank Age Group",
                       "Name", "Nation", "Age Group", "Total Time"]
            df_div = pd.DataFrame(division_results, columns=columns)
            df_div.to_csv(file_path_div, index=False)
            print(f"   Saved {len(df_div)} rows to {file_path_div}")
            human_pause(1, 2)

print("\nALL DONE.")
driver.quit()


STARTING Season_7

Found 73 races in Season_7
   Skipping 2025 Shanghai, all divisions already downloaded
   Skipping 2025 Atlanta, all divisions already downloaded
   Skipping 2025 Valencia, all divisions already downloaded
   Skipping 2025 Maastricht, all divisions already downloaded
   Skipping 2025 Mumbai, all divisions already downloaded
   Skipping 2025 World Championships, all divisions already downloaded
   Skipping 2025 Cardiff, all divisions already downloaded
   Skipping 2025 New York, all divisions already downloaded
   Skipping 2025 Riga, all divisions already downloaded
   Skipping 2025 Rimini, all divisions already downloaded
   Skipping 2025 Bangkok, all divisions already downloaded
   Skipping 2025 Berlin, all divisions already downloaded
   Skipping 2025 Incheon, all divisions already downloaded
   Skipping 2025 Heerenveen, all divisions already downloaded
   Skipping 2025 London, all divisions already downloaded
   Skipping 2025 Barcelona, all divisions already down

StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=145.0.7632.76); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#staleelementreferenceexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff66ae0aa55
	0x7ff66ab68630
	0x7ff66a8fd75d
	0x7ff66a9056c1
	0x7ff66a9087e4
	0x7ff66a9088af
	0x7ff66a9543b0
	0x7ff66a9819da
	0x7ff66a94aca6
	0x7ff66a9a591c
	0x7ff66a949098
	0x7ff66a949f83
	0x7ff66ae37810
	0x7ff66ae31afd
	0x7ff66ae52c1a
	0x7ff66ab83345
	0x7ff66ab8b81c
	0x7ff66ab71924
	0x7ff66ab71ad6
	0x7ff66ab57e47
	0x7ff8b1bae8d7
	0x7ff8b33cc40c


In [47]:
import time
import random
import pandas as pd
import os

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# ==========================
# CONFIG
# ==========================

BASE_URLS = {
    "Season_7": "https://results.hyrox.com/season-7/?pid=start",
    "Season_8": "https://results.hyrox.com/season-8/"
}

SAVE_ROOT = r"Datasets\Hyrox"

def human_pause(a=1, b=2):
    time.sleep(random.uniform(a, b))

# ==========================
# DRIVER
# ==========================

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 20)

# ==========================
# SCRAPER FUNCTION
# ==========================

def scrape_pages(race_name, division, gender, race_results):
    page_number = 1
    is_doubles = "DOUBLES" in division.upper()

    while True:
        try:
            wait.until(EC.presence_of_element_located(
                (By.CSS_SELECTOR, f"div[data-sex='{gender[0]}']")))
        except TimeoutException:
            return

        container = driver.find_element(
            By.CSS_SELECTOR, f"div[data-sex='{gender[0]}']")

        rows = container.find_elements(
            By.CSS_SELECTOR, "li.list-group-item.row")

        for row in rows:
            if "list-group-header" in row.get_attribute("class"):
                continue

            try:
                rank = row.find_element(By.CSS_SELECTOR, ".place-primary").text
                age_rank = row.find_element(By.CSS_SELECTOR, ".place-secondary").text
                total_time = row.find_element(By.CSS_SELECTOR, ".type-time").text.replace("Total", "").strip()
                age_group = row.find_element(By.CSS_SELECTOR, ".type-age_class").text.replace("Age Group", "").strip()

                if is_doubles:
                    members = row.find_elements(By.CSS_SELECTOR, ".type-relay_member a")
                    name = " & ".join([m.text for m in members])
                    nation = ""
                else:
                    name = row.find_element(By.CSS_SELECTOR, "h4.type-fullname").text
                    nation = row.find_element(By.CSS_SELECTOR, ".nation__abbr").text

                race_results.append([
                    race_name, division, gender,
                    rank, age_rank, name, nation,
                    age_group, total_time
                ])
            except:
                continue

        print(f"      Page {page_number} scraped")

        try:
            next_button = driver.find_element(By.XPATH, "//a[text()='>']")
            driver.execute_script("arguments[0].click();", next_button)
            page_number += 1
            human_pause(1, 2)
        except NoSuchElementException:
            print("      Last page reached")
            break

# ==========================
# MAIN LOOP
# ==========================

for season, base_url in BASE_URLS.items():

    print("\n==============================")
    print(f"STARTING {season}")
    print("==============================\n")

    season_folder = os.path.join(SAVE_ROOT, season)
    os.makedirs(season_folder, exist_ok=True)

    existing_files = os.listdir(season_folder)

    driver.get(base_url)
    human_pause(2, 3)

    wait.until(EC.presence_of_element_located(
        (By.ID, "default-lists-event_main_group")))

    race_dropdown = Select(
        driver.find_element(By.ID, "default-lists-event_main_group"))

    races = [(opt.text, opt.get_attribute("value"))
             for opt in race_dropdown.options]

    print(f"Found {len(races)} races in {season}")

    for race_name, race_value in races:

        safe_name = race_name.replace(" ", "_").replace("/", "-")

        # FAST CHECK: count how many CSVs start with race name
        matching_files = [
            f for f in existing_files
            if f.startswith(safe_name + "_") and f.endswith(".csv")
        ]

        if len(matching_files) >= 2:
            print(f"   Skipping {race_name}, already has 2 CSV files")
            continue

        print(f"\nProcessing race: {race_name}")

        # Now open race page only if needed
        driver.get(base_url)
        human_pause(1, 2)

        Select(driver.find_element(
            By.ID, "default-lists-event_main_group")
        ).select_by_value(race_value)

        human_pause(1, 2)

        select_division = Select(
            driver.find_element(By.ID, "default-lists-event"))

        available_divisions = [
            o.text.strip()
            for o in select_division.options
            if o.text.strip() != ""
        ]

        # Only take first 2 divisions
        divisions_to_scrape = available_divisions[:2]

        for div in divisions_to_scrape:

            div_safe = div.replace(" ", "_").replace("/", "-")
            file_path = os.path.join(
                season_folder, f"{safe_name}_{div_safe}.csv")

            if os.path.exists(file_path):
                print(f"   {div} already exists, skipping")
                continue

            print(f"   Division: {div}")

            division_results = []

            for gender_code, gender_name in [("M", "Men"), ("W", "Women")]:

                print(f"      Gender: {gender_name}")

                driver.get(base_url)
                human_pause(1, 2)

                Select(driver.find_element(
                    By.ID, "default-lists-event_main_group")
                ).select_by_value(race_value)

                Select(driver.find_element(
                    By.ID, "default-lists-event")
                ).select_by_visible_text(div)

                Select(driver.find_element(
                    By.ID, "default-lists-sex")
                ).select_by_value(gender_code)

                Select(driver.find_element(
                    By.ID, "default-num_results")
                ).select_by_value("100")

                driver.find_element(
                    By.ID, "default-submit").click()

                human_pause(2, 3)

                scrape_pages(race_name, div, gender_name, division_results)

            df = pd.DataFrame(division_results, columns=[
                "Race", "Division", "Gender",
                "Rank Overall", "Rank Age Group",
                "Name", "Nation", "Age Group", "Total Time"
            ])

            df.to_csv(file_path, index=False)
            print(f"   Saved {len(df)} rows to {file_path}")

print("\nALL DONE.")
driver.quit()


STARTING Season_7

Found 73 races in Season_7
   Skipping 2025 Shanghai, already has 2 CSV files
   Skipping 2025 Atlanta, already has 2 CSV files
   Skipping 2025 Valencia, already has 2 CSV files
   Skipping 2025 Maastricht, already has 2 CSV files
   Skipping 2025 Mumbai, already has 2 CSV files
   Skipping 2025 World Championships, already has 2 CSV files
   Skipping 2025 Cardiff, already has 2 CSV files
   Skipping 2025 New York, already has 2 CSV files
   Skipping 2025 Riga, already has 2 CSV files

Processing race: 2025 Rimini
   Division: HYROX PRO - Friday
      Gender: Men
      Page 1 scraped
      Page 2 scraped
      Last page reached
      Gender: Women
   Saved 151 rows to Datasets\Hyrox\Season_7\2025_Rimini_HYROX_PRO_-_Friday.csv
   Division: HYROX PRO - Saturday
      Gender: Men
      Gender: Women
   Saved 0 rows to Datasets\Hyrox\Season_7\2025_Rimini_HYROX_PRO_-_Saturday.csv

Processing race: 2025 Bangkok
   Division: HYROX PRO - Saturday
      Gender: Men
      Pa

KeyboardInterrupt: 

In [ ]:
import time
import random
import pandas as pd
import os

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# ==========================
# CONFIG
# ==========================

BASE_URLS = {
    "Season_7": "https://results.hyrox.com/season-7/?pid=start",
    "Season_8": "https://results.hyrox.com/season-8/"
}

SAVE_ROOT = r"Datasets\Hyrox"

def human_pause(a=3, b=7):
    time.sleep(random.uniform(a, b))

# ==========================
# DRIVER
# ==========================

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/120.0.0.0 Safari/537.36"
)

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 25)

# ==========================
# SCRAPE PAGES FUNCTION (UNCHANGED)
# ==========================

def scrape_pages(race_name, division, gender, race_results):
    page_number = 1
    is_doubles = "DOUBLES" in division.upper()

    while True:
        try:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f"div[data-sex='{gender[0]}']")))
        except TimeoutException:
            print("      No results container found")
            return

        human_pause(2, 5)

        try:
            container = driver.find_element(By.CSS_SELECTOR, f"div[data-sex='{gender[0]}']")
        except NoSuchElementException:
            print("      No container for this gender")
            return

        rows = container.find_elements(By.CSS_SELECTOR, "li.list-group-item.row")
        if not rows:
            print("      No rows found in container")
            return

        for row in rows:
            if "list-group-header" in row.get_attribute("class"):
                continue
            try:
                rank = row.find_element(By.CSS_SELECTOR, ".place-primary").text
                age_rank = row.find_element(By.CSS_SELECTOR, ".place-secondary").text
                total_time = row.find_element(By.CSS_SELECTOR, ".type-time").text.replace("Total", "").strip()
                age_group = row.find_element(By.CSS_SELECTOR, ".type-age_class").text.replace("Age Group", "").strip()

                if is_doubles:
                    members = row.find_elements(By.CSS_SELECTOR, ".type-relay_member a")
                    member_names = " & ".join([m.text for m in members])
                    race_results.append([race_name, division, gender, rank, age_rank, member_names, "", age_group, total_time])
                else:
                    name = row.find_element(By.CSS_SELECTOR, "h4.type-fullname").text
                    nation = row.find_element(By.CSS_SELECTOR, ".nation__abbr").text
                    race_results.append([race_name, division, gender, rank, age_rank, name, nation, age_group, total_time])
            except:
                continue

        print(f"      Page {page_number} scraped")
        human_pause(3, 6)

        try:
            next_button = driver.find_element(By.XPATH, "//a[text()='>']")
            driver.execute_script("arguments[0].click();", next_button)
            page_number += 1
            human_pause(5, 10)
        except NoSuchElementException:
            print("      Last page reached")
            break

# ==========================
# MAIN LOOP
# ==========================

for season, base_url in BASE_URLS.items():
    print(f"\n==============================\nSTARTING {season}\n==============================\n")

    season_folder = os.path.join(SAVE_ROOT, season)
    os.makedirs(season_folder, exist_ok=True)

    driver.get(base_url)
    human_pause(5, 10)

    try:
        wait.until(EC.presence_of_element_located((By.ID, "default-lists-event_main_group")))
    except TimeoutException:
        print(f"No races found for {season}")
        continue

    race_dropdown = Select(driver.find_element(By.ID, "default-lists-event_main_group"))
    races = [(opt.text, opt.get_attribute("value")) for opt in race_dropdown.options]

    print(f"Found {len(races)} races in {season}")

    for race_name, race_value in races:

        safe_name = race_name.replace(" ", "_").replace("/", "-")

        # ==========================
        # 🚀 FAST RACE SKIP
        # If 2 CSVs start with race name → skip race
        # ==========================
        existing_files = [f for f in os.listdir(season_folder) if f.startswith(safe_name)]
        if len(existing_files) >= 2:
            print(f"Skipping {race_name}, already downloaded")
            continue

        print(f"\nProcessing race: {race_name}")

        # Load race once to inspect divisions
        driver.get(base_url)
        human_pause(4, 8)

        Select(driver.find_element(By.ID, "default-lists-event_main_group")).select_by_value(race_value)
        human_pause(3, 6)

        select_division = Select(driver.find_element(By.ID, "default-lists-event"))
        available_divisions = [o.text.strip() for o in select_division.options]

        divisions_to_scrape = []

        # ==========================
        # PRO LOGIC
        # ==========================
        if "HYROX PRO" in available_divisions:
            divisions_to_scrape.append("HYROX PRO")
        elif "HYROX PRO - Overall" in available_divisions:
            divisions_to_scrape.append("HYROX PRO - Overall")
        else:
            pro_variants = [d for d in available_divisions if d.startswith("HYROX PRO -")]
            divisions_to_scrape.extend(pro_variants)

        # ==========================
        # DOUBLES LOGIC
        # ==========================
        if "HYROX PRO DOUBLES" in available_divisions:
            divisions_to_scrape.append("HYROX PRO DOUBLES")
        elif "HYROX PRO DOUBLES - Overall" in available_divisions:
            divisions_to_scrape.append("HYROX PRO DOUBLES - Overall")
        else:
            doubles_variants = [d for d in available_divisions if d.startswith("HYROX PRO DOUBLES -")]
            divisions_to_scrape.extend(doubles_variants)

        if not divisions_to_scrape:
            print("   No valid HYROX PRO divisions found")
            continue

        # ==========================
        # LOOP DIVISIONS (UNCHANGED STRUCTURE)
        # ==========================
        for div in divisions_to_scrape:

            div_safe_name = div.replace(" ", "_").replace("/", "-")
            file_path_div = os.path.join(season_folder, f"{safe_name}_{div_safe_name}.csv")

            if os.path.exists(file_path_div):
                print(f"   Division already downloaded: {div}")
                continue

            print(f"   Division: {div}")
            division_results = []

            for gender_code, gender_name in [("M", "Men"), ("W", "Women")]:

                print(f"      Gender: {gender_name}")

                driver.get(base_url)
                human_pause(4, 8)

                Select(driver.find_element(By.ID, "default-lists-event_main_group")).select_by_value(race_value)
                human_pause(2, 5)

                Select(driver.find_element(By.ID, "default-lists-event")).select_by_visible_text(div)
                human_pause(2, 5)

                Select(driver.find_element(By.ID, "default-lists-sex")).select_by_value(gender_code)
                Select(driver.find_element(By.ID, "default-num_results")).select_by_value("100")
                human_pause(2, 5)

                driver.find_element(By.ID, "default-submit").click()
                human_pause(5, 10)

                scrape_pages(race_name, div, gender_name, division_results)
                human_pause(2, 5)

            columns = ["Race", "Division", "Gender", "Rank Overall", "Rank Age Group",
                       "Name", "Nation", "Age Group", "Total Time"]

            df_div = pd.DataFrame(division_results, columns=columns)

            if len(df_div) > 0:
                df_div.to_csv(file_path_div, index=False)
                print(f"   Saved {len(df_div)} rows to {file_path_div}")
            else:
                print("   No rows scraped for this division")

            human_pause(3, 6)

print("\nALL DONE.")
driver.quit()


STARTING Season_7

Found 73 races in Season_7
Skipping 2025 Shanghai, already downloaded
Skipping 2025 Atlanta, already downloaded

Processing race: 2025 Valencia
   Division already downloaded: HYROX PRO - Overall
   Division: HYROX PRO DOUBLES - Overall
      Gender: Men
      Page 1 scraped
      Last page reached
      Gender: Women
      Page 1 scraped
      Last page reached
   No rows scraped for this division
Skipping 2025 Maastricht, already downloaded
Skipping 2025 Mumbai, already downloaded

Processing race: 2025 World Championships
   Division: HYROX PRO - Overall
      Gender: Men
      Page 1 scraped
      Last page reached
      Gender: Women
      Page 1 scraped
      Last page reached
   No rows scraped for this division
   Division: HYROX PRO DOUBLES - Overall
      Gender: Men
      Page 1 scraped
      Last page reached
      Gender: Women
      Page 1 scraped
      Last page reached
   No rows scraped for this division
Skipping 2025 Cardiff, already downloaded
Ski